In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
import pickle

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, Flatten, BatchNormalization, Conv1D, MaxPooling1D
from keras import Input
# from keras.layers import Conv1D, MaxPooling1D

from keras.metrics import mean_squared_error
import tensorflow as tf

with open('./data/floatimg.pkl', 'rb') as f:
    df = pickle.load(f)

n = 100 # 100일간 데이터 input
test_num = 10 # 10일간 데이터 output

df_forplot = df[-test_num:]
df = df[:-test_num]

# ---- Scaling ----

scaler = MinMaxScaler()
temp_data_scaled = scaler.fit_transform(df['value'].values.reshape(-1, 1))[:, 0]
df['value'] = np.float32(temp_data_scaled).reshape(-1, 1)

# -----------------

data = df.values

# 10일 뒤 데이터 예측
X = np.array([np.array(data[i:i+n]) for i in range(len(data)-n-test_num+1)])
y = np.array([np.array(df['value'].values[i+n+test_num-1]) for i in range(len(data)-n-test_num+1)])

# X = X.reshape((-1, n, 1))
# y = y.reshape((-1, 10, 1))

def plot_result(his):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
    ax1.plot(his.history['loss'])
    ax1.plot(his.history['val_loss'])
    ax1.set_title('Model Loss')
    ax1.set_ylabel('Loss')
    ax1.set_xlabel('Epoch')
    ax1.legend(['Train', 'Validation'], loc='upper right')

    ax2.plot(his.history['mae'])
    ax2.plot(his.history['val_mae'])
    ax2.set_title('Model MAE')
    ax2.set_ylabel('MAE')
    ax2.set_xlabel('Epoch')
    ax2.legend(['Train', 'Validation'], loc='upper right')

    plt.show()

def make_pred(num, model):
    data_pred = np.array(df['value'].values[:-n])
    for i in range(num):
        temp = data_pred[-n:]
        pred = model.predict(temp.reshape(1, -1))
        data_pred = np.append(data_pred, pred)
    return data_pred[-num:]

from keras.callbacks import EarlyStopping

custom_early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

X_train, X_test, y_train, y_test = train_test_split(np.array(X), np.array(y), test_size=0.2, random_state=42)

mse = tf.keras.losses.MeanSquaredError()
mae = tf.keras.losses.MeanAbsoluteError()

In [2]:
with open('./data/arrayed_img.pkl', 'rb') as f:
    df_a = pickle.load(f)

In [82]:
df_a.iloc[:, 0][0].shape

(512, 512, 1)

In [43]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((2144, 100, 3), (2144,), (536, 100, 3), (536,))

In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, Flatten, Dense, Concatenate, MaxPool2D
from tensorflow.keras.models import Model

# Parameters
image_shape = (512, 512, 1)  # Image dimensions
num_float_data = 1           # Number of float data points
output_length = 10           # Output length from each image processing CNN

# Function to create a CNN model for image processing
def create_image_cnn(input_shape, output_length, name):
    model_ = Sequential(name=f'{name}_cnn')
    model_.add(Conv2D(256, (3, 3), activation='relu', input_shape=input_shape))
    model_.add(BatchNormalization())
    model_.add(MaxPool2D(pool_size=(2, 2)))

    model_.add(Conv2D(128, (3, 3), activation='relu'))
    model_.add(BatchNormalization())
    model_.add(MaxPool2D(pool_size=(2, 2)))

    model_.add(Conv2D(64, (3, 3), activation='relu'))
    model_.add(BatchNormalization())
    model_.add(MaxPool2D(pool_size=(2, 2)))

    model_.add(Flatten())
    model_.add(Dense(256, activation='relu'))
    model_.add(Dense(128, activation='relu'))
    model_.add(Dense(64, activation='relu'))
    model_.add(Dense(32, activation='relu'))
    model_.add(Dense(output_length, activation='relu'))
    return model_

# 1. CNNs for Each Image
cnn1 = create_image_cnn(image_shape, output_length, 'image1')
cnn2 = create_image_cnn(image_shape, output_length, 'image2')

# 2. Inputs
float_input = Input(shape=(num_float_data,), name='float_input')
image1_input = Input(shape=image_shape, name='image1_input')
image2_input = Input(shape=image_shape, name='image2_input')

# Process each image
image1_features = cnn1(image1_input)
image2_features = cnn2(image2_input)

# Concatenation of Image Features and Float Data
concatenated = Concatenate()([float_input, image1_features, image2_features])

# 3. 1D CNN for Concatenated Data
x = Dense(128, activation='relu')(concatenated)
x = Dense(64, activation='relu')(x)
x = Dense(64, activation='relu')(x)
x = Dense(32, activation='relu')(x)
final_output = Dense(1, activation='relu', name='final_output')(x)  # Assuming a single float output

# Model
model1 = Model(inputs=[float_input, image1_input, image2_input], outputs=final_output)

model1.compile(loss='mse', optimizer='adam', metrics=['mae'])

model1.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 image1_input (InputLayer)      [(None, 512, 512, 1  0           []                               
                                )]                                                                
                                                                                                  
 image2_input (InputLayer)      [(None, 512, 512, 1  0           []                               
                                )]                                                                
                                                                                                  
 float_input (InputLayer)       [(None, 1)]          0           []                               
                                                                                              

In [45]:
float_train = X_train[:, :, 0].reshape(-1, 100, 1)

In [46]:
print(type(float_train[0][0][0]))

<class 'float'>


In [47]:
float_train.shape

(2144, 100, 1)

In [48]:
for i in range(2144):
    for j in range(100):
        float_train[i][j][0] = np.float32(float_train[i][j][0])

In [5]:
y_train

array([0.25157562, 0.22531512, 0.4590336 , ..., 0.3902311 , 0.7079832 ,
       0.33981094], dtype=float32)

In [ ]:
# float_train.shape, float_test.shape

In [65]:
X_train[:, :, 1].shape

(2144, 100)

In [74]:
type(X_train[:, :, 1][0][0])

numpy.ndarray

In [84]:
X_train[:, :, 1][0][0].shape

(512, 512, 1)

In [85]:
ff = np.asarray(X_train[:, :, 1])

In [8]:
fu = np.concatenate([np.concatenate([X_train[:, :, 1][i][j] for j in range(100)])[np.newaxis, :] for i in range(2144)])

MemoryError: Unable to allocate 100. MiB for an array with shape (51200, 512, 1) and data type float32

In [3]:
ffuck = np.stack(X_train[:, :, 1], axis=-1)

In [4]:
ffuck.shape

(100, 2144)

In [5]:
ar = np.array([[1, 2], [3, 4]])

In [7]:
ar.

In [92]:
ffuck_real = ffuck.reshape((2144, 100, 512, 512, 1))

ValueError: cannot reshape array of size 26214400 into shape (2144,100,512,512,1)

In [ ]:
fu.shape

In [72]:
fuck = np.array([np.array([np.array([np.array([np.array([X[:, :, 1][i][j][k][l][m] for m in range(512)])for l in range(512)]) for k in range(1)]) for j in range(100)]) for i in range(2144)])

IndexError: index 1 is out of bounds for axis 0 with size 1

In [69]:
np.array(fuck).shape

(2144, 100, 3)

In [ ]:
np.array([X_train])

In [3]:
X_train[:, :, 1].shape

(2144, 100)

In [7]:
X_train[:, :, 1][0][0][10][10][0]

0.0

In [2]:
img1_train = [[[[[[X_train[:, :, 1][i][j][k][l][m]] for m in range(1)] for l in range(512)] for k in range(512)] for j in range(100)] for i in range(2144)] # (2144, 512, 512, 1)

In [ ]:
with open('./data/img1_train.pkl', 'wb') as f:
    pickle.dump(img1_train, f)

In [ ]:
img2_train = [[[[[[X_train[:, :, 2][i][j][k][l][m]] for m in range(1)] for l in range(512)] for k in range(512)] for j in range(100)] for i in range(2144)] # (2144, 512, 512, 1)
with open('./data/img2_train.pkl', 'wb') as f:
    pickle.dump(img2_train, f)

In [57]:
img1_train.shape

(2144, 100, 512, 1)

In [1]:
img2_train = np.array([np.array([np.float32(X_train[j, :, 2][0][i]) for i in range(len(X_train[j, :, 2][0]))]) for j in range(len(X_train))]) # (2144, 512, 512, 1)

SyntaxError: closing parenthesis ')' does not match opening parenthesis '[' (699834090.py, line 1)

In [7]:
img1_train = np.array([[[np.array([np.float32(X_train[j, :, 1][0][m][i]) for i in range(len(X_train[j, :, 1][0]))]) for m in range(512)] for k in range(100)] for j in range(len(X_train))]) # (2144, 512, 512, 1)

In [15]:
img1_train = np.array([[[np.array([np.float32(X_train[j, :, 1][k][m][i]) for i in range(len(X_train[j, :, 1][k]))]) for m in range(512)] for k in range(100)] for j in range(len(X_train))]) # (2144, 512, 512, 1)

In [13]:
X_train[:, :, 1][:, 10].shape

(2144,)

In [8]:
img1_train.shape

(2144, 512, 512, 1)

In [9]:
# img1_test = np.array([np.array([X_test[j, :, 1][0][i] for i in range(len(X_test[j, :, 1][0]))]) for j in range(len(X_test))]) # (536, 512, 512, 1)

In [10]:
# img2_test = np.array([np.array([X_test[j, :, 2][0][i] for i in range(len(X_test[j, :, 2][0]))]) for j in range(len(X_test))]) # (536, 512, 512, 1)

In [23]:
type(img2_train[0][0][0])==np.float32

False

In [24]:
img2_train.shape

(2144, 512, 512, 1)

In [26]:
for i, j, k in range(2144, 512, 512):
    print(type(img1_train[i][j][k]))

In [29]:
float_train.shape

(2144, 100, 1)

In [32]:
print(type(float_train[0][0][0]))

<class 'float'>


In [49]:
float_train = np.asarray(float_train, dtype=np.float32)
img1_train = np.asarray(img1_train, dtype=np.float32)
img2_train = np.asarray(img2_train, dtype=np.float32)

In [71]:
img2_train.shape

(2144, 512, 512, 1)

In [51]:
model1.fit([float_train, img1_train, img2_train], y_train, epochs=10, batch_size=32, validation_split=0.2, callbacks=[custom_early_stopping], verbose=1)

Epoch 1/10


ValueError: in user code:

    File "c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1023, in train_step
        y_pred = self(x, training=True)
    File "c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\Users\User\AppD